# Ghost Supply - Quick Start Demo

This notebook demonstrates the core functionality of Ghost Supply:
1. Loading terrain and threat data
2. Building routing graph
3. Running CVaR optimization
4. Comparing with baselines
5. Visualizing results

In [ ]:
# Setup
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from ghost_supply.utils.data_loader import DataLoader
from ghost_supply.utils.constants import STUDY_AREA_BOUNDS
from ghost_supply.perception.terrain import TerrainAnalyzer
from ghost_supply.perception.threat_model import ThreatPredictor
from ghost_supply.perception.weather import WeatherModel
from ghost_supply.decision.graph_builder import GraphBuilder
from ghost_supply.decision.cvar_routing import CVaRRouter
from ghost_supply.decision.facility_location import generate_candidate_depots, select_depots

print("✓ Imports successful")

## Step 1: Load/Generate Data

In [ ]:
# Initialize data loader
loader = DataLoader()

# Load or generate terrain
print("Loading terrain data...")
dem_result = loader.load_dem()

if dem_result is None:
    print("No DEM found. Generating synthetic terrain...")
    elevation, transform = loader.create_synthetic_dem(STUDY_AREA_BOUNDS, save=True)
else:
    elevation, transform = dem_result

print(f"✓ Terrain loaded: {elevation.shape}")

# Visualize elevation
plt.figure(figsize=(10, 8))
plt.imshow(elevation, cmap='terrain')
plt.colorbar(label='Elevation (m)')
plt.title('Digital Elevation Model - Pokrovsk Region')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

## Step 2: Initialize Threat Model

In [ ]:
# Initialize threat predictor
print("Generating threat data...")
threat_predictor = ThreatPredictor()

# Load or generate incidents
incidents = loader.load_incidents()
if incidents is None:
    incidents = threat_predictor.generate_synthetic_incidents(num_incidents=500, days_history=180)
    loader.save_incidents(incidents)

print(f"✓ Loaded {len(incidents)} incidents")

# Train temporal model
print("Training Prophet model...")
threat_predictor.train_temporal_model(incidents)
print("✓ Temporal model trained")

# Identify kill zones
kill_zones = threat_predictor.identify_kill_zones()
print(f"✓ Identified {len(kill_zones)} kill zones")

# Visualize incidents
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(incidents['longitude'], incidents['latitude'], 
            c=incidents['casualties'], cmap='Reds', alpha=0.5, s=10)
plt.colorbar(label='Casualties')
plt.title('Incident Locations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.subplot(1, 2, 2)
incidents_by_hour = incidents.groupby(incidents['timestamp'].dt.hour).size()
plt.bar(incidents_by_hour.index, incidents_by_hour.values)
plt.title('Incidents by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Count')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Step 3: Build Routing Graph

In [ ]:
# Initialize terrain analyzer
terrain = TerrainAnalyzer(elevation, transform, STUDY_AREA_BOUNDS)

# Build graph
print("Building routing graph from OSM...")
graph_builder = GraphBuilder(terrain, threat_predictor, WeatherModel())
graph = graph_builder.build_from_osm(STUDY_AREA_BOUNDS)

print(f"✓ Graph built: {len(graph.nodes)} nodes, {len(graph.edges)} edges")

# Select depots
frontline_lat = (STUDY_AREA_BOUNDS['north'] + STUDY_AREA_BOUNDS['south']) / 2
frontline_lon = (STUDY_AREA_BOUNDS['east'] + STUDY_AREA_BOUNDS['west']) / 2

candidates = generate_candidate_depots(STUDY_AREA_BOUNDS, frontline_lat, num_candidates=20)
depots = select_depots(candidates, frontline_lat, frontline_lon, num_depots=3)

# Add custom nodes
depot_positions = [(d.latitude, d.longitude) for d in depots]
frontline_positions = [
    (frontline_lat + 0.02, frontline_lon - 0.1),
    (frontline_lat + 0.01, frontline_lon),
    (frontline_lat - 0.01, frontline_lon + 0.1),
]

graph_builder.add_custom_nodes(depot_positions, frontline_positions)

print(f"✓ Added {len(depots)} depots and {len(frontline_positions)} frontline positions")

## Step 4: Enrich Graph with Tactical Data

In [ ]:
# Calculate viewshed from enemy positions
print("Computing viewshed...")
observer_positions = [(frontline_lat + 0.05, frontline_lon - 0.05), 
                      (frontline_lat + 0.03, frontline_lon + 0.05)]
viewshed = terrain.calculate_viewshed(observer_positions)

# Enrich graph
departure_time = datetime.now().replace(hour=14, minute=0)
graph_builder.enrich_graph(
    viewshed=viewshed,
    rf_coverage=None,
    weather='clear',
    timestamp=departure_time
)

print("✓ Graph enriched with tactical attributes")

## Step 5: Run CVaR Optimization

In [ ]:
# Get origin and destination nodes
depot_nodes = [n for n, d in graph.nodes(data=True) if d.get('node_type') == 'depot']
frontline_nodes = [n for n, d in graph.nodes(data=True) if d.get('node_type') == 'frontline']

origin = depot_nodes[0]
destination = frontline_nodes[0]

print(f"Route: Depot {origin} → Frontline {destination}")

# Initialize router
router = CVaRRouter(graph, alpha=0.95, num_scenarios=100)

# Run optimizations
print("\nOptimizing routes...")
print("  1. CVaR 95%...")
cvar_route = router.optimize(origin, destination, cargo_value=7.0)

print("  2. Shortest distance...")
shortest_route = router.shortest_distance(origin, destination)

print("  3. Fastest time...")
fastest_route = router.shortest_time(origin, destination)

print("  4. Mean risk...")
mean_risk_route = router.mean_risk(origin, destination, cargo_value=7.0)

print("\n✓ All routes computed")

## Step 6: Compare Results

In [ ]:
import pandas as pd

# Create comparison table
results = {
    'Method': ['CVaR 95%', 'Shortest Distance', 'Fastest Time', 'Mean Risk'],
    'Time (min)': [
        cvar_route.time_minutes,
        shortest_route.time_minutes,
        fastest_route.time_minutes,
        mean_risk_route.time_minutes
    ],
    'Distance (km)': [
        cvar_route.distance_km,
        shortest_route.distance_km,
        fastest_route.distance_km,
        mean_risk_route.distance_km
    ],
    'Mean Risk': [
        cvar_route.mean_risk,
        shortest_route.mean_risk,
        fastest_route.mean_risk,
        mean_risk_route.mean_risk
    ],
    'CVaR 95%': [
        cvar_route.cvar_95,
        shortest_route.cvar_95,
        fastest_route.cvar_95,
        mean_risk_route.cvar_95
    ]
}

df = pd.DataFrame(results)
print("\n" + "="*80)
print("ROUTE COMPARISON")
print("="*80)
print(df.to_string(index=False))
print("="*80)

# Calculate gains
time_overhead = (cvar_route.time_minutes - shortest_route.time_minutes) / shortest_route.time_minutes * 100
risk_reduction = (shortest_route.cvar_95 - cvar_route.cvar_95) / shortest_route.cvar_95 * 100

print(f"\nCVaR vs Shortest Distance:")
print(f"  Time overhead: +{time_overhead:.1f}%")
print(f"  Risk reduction: -{risk_reduction:.1f}%")
print(f"  Survival prob: {(1-shortest_route.cvar_95)*100:.1f}% → {(1-cvar_route.cvar_95)*100:.1f}%")

## Step 7: Visualize Routes

In [ ]:
# Extract coordinates
cvar_lats = [p[0] for p in cvar_route.path]
cvar_lons = [p[1] for p in cvar_route.path]

shortest_lats = [p[0] for p in shortest_route.path]
shortest_lons = [p[1] for p in shortest_route.path]

# Plot routes
plt.figure(figsize=(12, 10))

# Plot kill zones
for kz in kill_zones:
    circle = plt.Circle(
        (kz['center'][1], kz['center'][0]),
        kz['radius_km'] / 111,  # Rough conversion to degrees
        color='red', alpha=0.2, label='Kill Zone' if kz == kill_zones[0] else ''
    )
    plt.gca().add_patch(circle)

# Plot routes
plt.plot(shortest_lons, shortest_lats, 'r--', linewidth=2, label='Shortest Distance', alpha=0.7)
plt.plot(cvar_lons, cvar_lats, 'g-', linewidth=3, label='CVaR 95% (Optimized)', alpha=0.9)

# Plot start/end
plt.scatter(cvar_lons[0], cvar_lats[0], c='blue', s=200, marker='s', label='Depot', zorder=5)
plt.scatter(cvar_lons[-1], cvar_lats[-1], c='orange', s=200, marker='*', label='Frontline', zorder=5)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Ghost Supply Route Comparison')
plt.legend()
plt.grid(alpha=0.3)
plt.axis('equal')
plt.show()

## Step 8: Export Mission Package

In [ ]:
from ghost_supply.output.cot_export import export_mission_package
from ghost_supply.output.report import generate_mission_briefing

# Export ATAK package
print("Generating mission package...")
package_path = export_mission_package(
    cvar_route,
    shortest_route,
    kill_zones,
    '../outputs/demo_mission.zip',
    'Ghost Supply Demo Mission'
)

print(f"✓ Mission package: {package_path}")

# Generate briefing
briefing = generate_mission_briefing(
    cvar_route,
    shortest_route,
    'clear',
    departure_time,
    'munitions',
    9,
    kill_zones
)

print("\n" + briefing[:500] + "...")
print("\n[Full briefing truncated for notebook display]")

## Conclusion

This demo showed:

1. ✓ Terrain and threat data generation
2. ✓ Graph construction and enrichment
3. ✓ CVaR optimization outperforms baselines
4. ✓ Visualization of routes and kill zones
5. ✓ Export to ATAK-compatible formats

**Key Takeaway**: CVaR routing reduces tail risk significantly while maintaining reasonable time overhead.

For more advanced features, see:
- `02_pareto_analysis.ipynb` - Multi-objective optimization
- `03_game_theory.ipynb` - Stackelberg equilibrium
- `04_rf_propagation.ipynb` - RF coverage modeling
- `streamlit run ../app/streamlit_app.py` - Interactive dashboard